# Lab 2:  Problem 3

In [ ]:
from IPython.core.display import HTML
def css_styling():
    styles = open("../styles/tma4215.css", "r").read()
    return HTML(styles)

# Comment out next line and execute this cell to restore the default notebook style 
css_styling()

We will now use Newton's method to find the roots to a *complex* function, namely the function
    $$f_C(z) = z^3 - 1.$$
### a)
Write the complex function $f_C: \mathbb{C} \to \mathbb{C}$ as a real vector-valued function $\mathbf{f}: \mathbb{R}^2 \to \mathbb{R}^2$ and write down the Jacobian.  
(*Hint: use rectangular coordinates $z = x_1+\mathrm{i}x_2$)*

<font color='blue'>
    
   Solution: 

</font>

Let $z = x_1 + i x_2$, then
\begin{align}
f(z) &= (x_1 + i x_2)^3 - 1 \\
&= x_{1}^{3} + 3 i x_{1}^{2} x_{2} - 3 x_{1} x_{2}^{2} - i x_{2}^{3} - 1 \\
&= (x_{1}^{3} - 3 x_{1} x_{2}^{2} - 1 ) + i(3 x_{1}^{2} x_{2}- x_{2}^{3})
\end{align}
Define a real vector-valued function $\mathbf{f}: \mathbb{R}^2 \to \mathbb{R}^2$, with $\mathbf{x} = x_1 + ix_2$, then:
\begin{equation}
\mathbf{f}(\mathbf{x}) = (x_{1}^{3} - 3 x_{1} x_{2}^{2} - 1 , 3 x_{1}^{2} x_{2}- x_{2}^{3})
\end{equation}





The Jacobian is then 

$$
d\mathbf{f} = 
\begin{pmatrix} 
3x_1^2 - 3x_2^2 &-6x_1 x_2 \\
6x_1 x_2 & 3x_1^2 - 3x_2^2
\end{pmatrix}
$$

### b)
Write a function $\texttt{Newton}$ which performs Newton iteration until $\|\mathbf{f}(\mathbf{x}^k)\|_p$ is smaller than some given tolerance $tol$. You can choose what $p$-norm $\|\cdot\|_p$ you use. The function should take as input parameters  
 - Initial guess $\mathbf{x}^{(0)}$
 - tolerance $tol$
 - Maximum number of iterations $itermax$.  

The function should return the final iterate $\mathbf{x}^{(k)}$, and some indication of whether the iteration converged. 

In [ ]:
import numpy as np
import scipy.linalg as la
import matplotlib.pyplot as plt
import numpy.linalg as la


def Newton(x0, tol, itermax, f, df, p=2):
    x_k = x0
    error = 2 * tol

    iteration = 0
    errors = []
    while error > tol:
        try:
            x_k1 = x_k - la.inv(df(x_k)) @ f(x_k)
        except la.LinAlgError:
            return x_k, False
        # error = la.norm(x_k - x_k1) # vi må prøve med en annen p-norm = [sum(abs(diff_i)**p)]**1/p
        error = np.sum((x_k - x_k1)**p) ** (1 / p)
        errors.append(error)

        iteration += 1
        x_k = x_k1

    return x_k, True


def jacobian(x):
    x_1, x_2 = x
    J = np.array([[3 * x_1 ** 2 - 3 * x_2 ** 2, - 6 * x_1 * x_2],
                  [6 * x_1 * x_2, 3 * x_1 * 2 - 3 * x_2 ** 2]]).reshape((2, 2))
    return J


def func(x):
    x_1, x_2 = x
    return np.array([x_1 ** 3 - 3 * x_1 * x_2 ** 2 - 1, 3 * x_1 ** 2 * x_2 - x_2 ** 3]).reshape((-1, 1))


"""
# Test for different p-norms.

x_0 = np.array([2, 1]).reshape(-1, 1)
for p in [1, 2, 4, 10, 50, 100, 200, 10000]:
    print("p =", p)
    res, conv = Newton(x_0, 1e-16, 100, func, jacobian, p)
    print(res)
"""

### c)
The function $\mathbf{f}$ has exactly three roots. (Which?) Hence, if the iteration converges it might converge to any of the three roots. You will now study the dependence on initial guess to which root the iteration converges. Pick $N$ equidistant values of $x^{(0)}_1$ in the interval $[-1,1]$ and $N$ equidistant values of $x^{(0)}_2$ in the interval $[-1,1]$. For each point $\mathbf{x}^{(0)} = (x^{(0)}_1, x^{(0)}_2)^T$, perform the newton iteration you have defined above. Give the point a color depending on whether the iteration converged in time. If the iteration converged, the point should get a different color depending on which point it converged to. Plot the result.

In [ ]:
N = 20

# r = 2
x_1 = np.linspace(-1, 1, N)
x_2 = np.linspace(-1, 1, N)
tol = 1e-5
itermax = 100

fig, axes = plt.subplots()

root_1 = np.array([1, 0]).reshape((-1, 1))
root_2 = np.array([-.5, np.sqrt(3)/2]).reshape((-1, 1))
root_3 = np.array([-.5, -np.sqrt(3)/2]).reshape((-1, 1))

roots = [root_1, root_2, root_3]

result = np.zeros((N, N))

with_start_offset = True

for i, v_1 in enumerate(x_1):
    for j, v_2 in enumerate(x_2):
        
        offset = np.array([1/(4*N), 1/(4*N)]) if with_start_offset else 0
        x = (np.array([i, j])  + offset).reshape((-1, 1))
        # x = (np.array([i, j])).reshape((-1, 1))
        x_k, converged = Newton(x, tol, itermax, func, jacobian)

        if converged:
            # result[j, i] = 1

            if la.norm(x_k - root_1) < tol:
                result[j, i] = 1
            elif la.norm(x_k - root_2) < tol:
                result[j, i ] = 2/3
            elif la.norm(x_k - root_3) < tol:
                result[j, i ] = 1/3
        else:
            result[j, i] = 0

axes.imshow(result, interpolation='none', extent=(-1, 1, -1, 1))
plt.show()


The three roots of $\mathbf{f}$ is $r_1 = 1$, $r_2 = -0.5 + i\frac{\sqrt{3}}{2}$ and $r_3 = -0.5 - i \frac{\sqrt{3}}{2}$.


In the figure produced by the code above, the lightest three colours correspond to $r_1$, $r_2$ and $r_3$ respectively. If there are four colours in the plot, then the darkest colour corresponds to points that didn't converge. After adding a small perturbation to all starting points, all points seemed to converge. 

We see from the plot that almost all starting points end up converging to $r_1$.


*Hint 1: You will encounter an error if you try with the initial guess $\mathbf{x}^{(0)} = \mathbf{0}$. It might be good to offset all initial guesses by some small perturbation $\mathbf{\delta}$.*     

*Hint 2: A good way to measure which root the iteration converged to is looking at the argument if the point as a complex number, that is $$\arg(x^{(k)}_1 + \mathrm{i}x^{(k)}_2).$$ Store the result in a $N\times N$ array. If the iteration did not converge, the point can be given the value* **None**. *The result can then be plotted using the matplotlib.pyplot function $\texttt{imshow}$.*

### d)
Discuss the following: Does it matter what $p$-norm you use in your Newton algorithm? What happens if you change norm? Does the result change qualitively? Remember that the $p$-norm on $\mathbb{R}^n$  is defined as
$$
\|\mathbf{x}\|_p := \left(\sum_{k=1}^n|x_k|^p\right)^{1/p}
$$
for $1<p<\infty$.

<font color='blue'>
    
   Higher norms seem to make the algorithm converge faster but does not seem to reach the correct points.
   
</font>

### e) (Not mandatory)
Modify your Newton function so that it returns the number of iterations used. Then, plot the number of iterations used for each initial guess $\mathbf{x}^{(0)}.$ What do you observe?

In [ ]:
#Write code here

### Remark
If you colored the plot according to the hint, the white parts are called the *Julia set* of the rational function $$Q(z) = z - \frac{f_C(z)}{f_C'(z)}.$$ The colored parts are called *Fatou components.* If you would like to more about the plot above and others like it, there is a link to the wikipedia page of [Julia sets](https://en.wikipedia.org/wiki/Julia_set).